In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/CS247/Models')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
!pip install surprise
import surprise
from surprise import *
from util import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
train_data = pd.read_pickle("/content/drive/MyDrive/CS247/Data/full_training_data.pkl")
train_data = merge_user_id(train_data, on='review_profilename').rename(columns={'id':'user_id'})
train_data = merge_beer_id(train_data, on='beer_name').rename(columns={'id':'beer_id'})[['user_id','beer_id','review_overall']]
test_data = pd.read_pickle("/content/drive/MyDrive/CS247/Data/test_data.pkl")
test_data = merge_user_id(test_data, on='review_profilename').rename(columns={'id':'user_id'})
test_data = merge_beer_id(test_data, on='beer_name').rename(columns={'id':'beer_id'})[['user_id','beer_id','review_overall']]

In [ ]:
train_data.head()

,user_id,beer_id,review_overall
0,0,0,4.0
1,1,0,4.5
2,2,0,4.0
3,3,0,4.0
4,5,0,4.5


# KNN basic model at K = 20


In [ ]:
## https://surprise.readthedocs.io/en/stable/FAQ.html
reader = Reader(rating_scale=(0,5))
training_dataset = Dataset.load_from_df(train_data, reader)
training_set = training_dataset.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': True, 'shrinkage':0}
model = algo = KNNWithMeans(
    k = 20, min_k = 1, sim_options=sim_options)
model.fit(training_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
reader = Reader(rating_scale=(0,5))
test_dataset = Dataset.load_from_df(test_data, reader)
testset = test_dataset.build_full_trainset()
testset = testset.build_testset()
predictions = model.test(testset)

surprise.accuracy.rmse(predictions, verbose=True)

RMSE: 0.6330


0.633011397814727

# precision & recall


In [ ]:
from collections import defaultdict
## https://surprise.readthedocs.io/en/stable/FAQ.html
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

# Precision and recall can then be averaged over all users
print("precision at 5: ", sum(prec for prec in precisions.values()) / len(precisions))
print("recall at 5:", sum(rec for rec in recalls.values()) / len(recalls))

precision at 5:  0.5912224999437159
recall at 5: 0.36811146166631636


### Random Recommender

In [ ]:
from surprise import AlgoBase
import random

class RandomRecommender(AlgoBase):
  def __init__(self):
    AlgoBase.__init__(self)

  def estimate(self, u, i):
    return random.uniform(0,5)

In [ ]:
random_recommender = RandomRecommender()
random_recommender.fit(training_set)
random_predictions = random_recommender.test(testset)
surprise.accuracy.rmse(random_predictions, verbose=True)

RMSE: 2.0788


2.078842631659123

In [ ]:
precisions, recalls = precision_recall_at_k(random_predictions, k=5, threshold=4)

# Precision and recall can then be averaged over all users
print("precision at 5: ", sum(prec for prec in precisions.values()) / len(precisions))
print("recall at 5:", sum(rec for rec in recalls.values()) / len(recalls))

precision at 5:  0.38581256051701196
recall at 5: 0.1570168918866258
